---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    towns = pd.read_csv("university_towns.txt", sep="\[edit\]", header=None, names=['State','RegionName'])
    towns['RegionName'] = towns['State'].apply(lambda x: x if re.search('\(', x) else '')
    towns['State'] = towns['State'].apply(lambda x: x if not re.search('\W', x) 
                                                      else np.NaN).ffill()
    towns.drop(towns[towns['RegionName']==''].index, inplace=True)
    towns['RegionName'] = towns['RegionName'].apply(lambda x: re.split(' \(',x)[0])
    
    return towns

get_list_of_university_towns().head()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo


In [5]:
gdplev = pd.read_excel('gdplev.xls', skiprows = 219, usecols = [4,6], names = ['yq', 'gdp_09'])
gdplev.iloc[33:40]

,yq,gdp_09
33,2008q2,14963.4
34,2008q3,14891.6
35,2008q4,14577.0
36,2009q1,14375.0
37,2009q2,14355.6
38,2009q3,14402.5
39,2009q4,14541.9


In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    #use for loops for this question or the shift() function of pandas.
    gdplev = pd.read_excel('gdplev.xls', skiprows = 219, usecols = [4,6], names = ['yq', 'gdp_09'])
    rec_start=""
    
    for index in range(len(gdplev)-1):
        if ((gdplev['gdp_09'].iloc[index]-gdplev['gdp_09'].iloc[index+1]) > 0 
            and (gdplev['gdp_09'].iloc[index+1]-gdplev['gdp_09'].iloc[index+2]) > 0):
            rec_start=gdplev['yq'].iloc[index]
            break
    
    return rec_start

get_recession_start()

'2008q2'

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdplev = pd.read_excel('gdplev.xls', skiprows = 219, usecols = [4,6], names = ['yq', 'gdp_09'])
    rec_end=""
    
    for index in range(gdplev[gdplev['yq']==get_recession_start()].index[0], len(gdplev)-1):
        if (gdplev['gdp_09'].iloc[index]-gdplev['gdp_09'].iloc[index+1] < 0 
            and gdplev['gdp_09'].iloc[index+1]-gdplev['gdp_09'].iloc[index+2] < 0):
            rec_end = gdplev['yq'].iloc[index+2]
            break
       
    return rec_end

get_recession_end()

'2009q4'

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    rec_start_index = gdplev[gdplev['yq']==get_recession_start()].index[0]
    rec_end_index = gdplev[gdplev['yq']==get_recession_end()].index[0]
    min_gdp = min(gdplev['gdp_09'].iloc[rec_start_index:rec_end_index])
    rec_bottom = gdplev['yq'][gdplev['gdp_09']==min_gdp].iloc[0]
    
    return rec_bottom
    
get_recession_bottom()

'2009q2'

In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    median_home_price = pd.read_csv('City_Zhvi_AllHomes.csv', 
                                    index_col=["State","RegionName"],
                                    usecols=(np.r_[1,2,51:251]))
    
    mean_q_home_price = (median_home_price.groupby(pd.PeriodIndex(median_home_price.columns, freq='Q'), axis=1)
                                          .mean()
                                          .rename(columns=lambda c: str(c).lower()))
    
    return mean_q_home_price

convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
CA,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
IL,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
PA,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
AZ,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [10]:
gdplev = pd.read_excel('gdplev.xls', skiprows = 219, usecols = [4,6], names = ['yq', 'gdp_09'])

    # keep only housing price columns corresponding to recession start & recession bottom
recession_housing = convert_housing_data_to_quarters()[[get_recession_start(), get_recession_bottom()]]
    
    # add third column - a ratio of the two (should be a smaller number for univ towns under current hypothesis - 
    #     they are less affected by recession ==> they keep growing)
recession_housing['ratio'] = recession_housing.iloc[:,0]/recession_housing.iloc[:,1]
    
    # map state initials to full state names
recession_housing = recession_housing.reset_index()
recession_housing['State'] = recession_housing['State'].map(states)
recession_housing = recession_housing.set_index(['State','RegionName']).groupby(level=[0,1]).mean()
#recession_housing

univ_housing = recession_housing.loc[get_list_of_university_towns().set_index(['State','RegionName']).index.tolist()]

#blop = recession_housing.reset_index().iloc[:,0:2].isin(univ_housing.reset_index().iloc[:,0:2])

#recession_housing.iloc[:,0:2][recession_housing.reset_index().iloc[:,0:2].isin(univ_housing.reset_index().iloc[:,0:2])==True]

#recession_housing.head()

recession_housing

2008q2         2009q2     ratio
State     RegionName                                             
Alabama   Adamsville        86066.666667   85766.666667  1.003498
          Alabaster        156933.333333  149666.666667  1.048552
          Albertville       87933.333333   84333.333333  1.042688
          Arab             111033.333333  115366.666667  0.962439
          Ardmore          149866.666667  143900.000000  1.041464
          Axis                       NaN  132700.000000       NaN
          Baileyton                  NaN            NaN       NaN
          Bay Minette      114066.666667  104666.666667  1.089809
          Bayou La Batre    60400.000000   65333.333333  0.924490
          Bessemer                   NaN            NaN       NaN
          Birmingham        72400.000000   68366.666667  1.058996
          Boaz              82766.666667   80166.666667  1.032432
          Brent            122533.333333  106866.666667  1.146600
          Brighton                   NaN            NaN       NaN
          Brookwood        117466.666667  115500.000000  1.017027
          Buhl             122066.666667  113700.000000  1.073585
          Calera           136200.000000  133066.666667  1.023547
          Center Point     103233.333333   94966.666667  1.087048
          Centreville      124100.000000  107166.666667  1.158009
          Chalkville       118700.000000  112833.333333  1.051994
          Chancellor                 NaN            NaN       NaN
          Chelsea          213233.333333  199066.666667  1.071165
          Chickasaw         78033.333333   72133.333333  1.081793
          Chunchula        137400.000000  131833.333333  1.042225
          Citronelle       106900.000000  104500.000000  1.022967
          Clay             151666.666667  141900.000000  1.068828
          Coden            114500.000000  113566.666667  1.008218
          Coker            142166.666667  139366.666667  1.020091
          Concord          107166.666667  103700.000000  1.033430
          Cottondale       132733.333333  126566.666667  1.048723
...                                  ...            ...       ...
Wisconsin Vandenbroek                NaN            NaN       NaN
          Vernon           257233.333333  251166.666667  1.024154
          Vienna           250866.666667  249766.666667  1.004404
          Vinland          198000.000000  187233.333333  1.057504
          Wales                      NaN            NaN       NaN
          Waterford        248266.666667  220366.666667  1.126607
          Waukesha         192666.666667  188200.000000  1.023734
          Waunakee         262466.666667  253966.666667  1.033469
          Waupun           112433.333333  106966.666667  1.051106
          Wausau            98500.000000   91600.000000  1.075328
          Wayne            245266.666667  233766.666667  1.049194
          West Allis       115733.333333  106300.000000  1.088743
          Weston           140533.333333  128700.000000  1.091945
          Wheatland        197933.333333  193466.666667  1.023088
          Whitelaw         126433.333333  122200.000000  1.034643
          Williams Bay     253866.666667  238633.333333  1.063836
          Wilson           175166.666667  183666.666667  0.955145
          Wind Lake        279833.333333  242833.333333  1.152368
          Wind Point       283666.666667  276433.333333  1.026167
          Wisconsin Dells  156800.000000  154066.666667  1.017741
          Wolf River       154533.333333  173833.333333  0.888974
          Woodruff                   NaN            NaN       NaN
          Woodville        154866.666667  168600.000000  0.918545
          Yorkville        283300.000000  264100.000000  1.072700
Wyoming   Bar Nunn         202466.666667  193166.666667  1.048145
          Burns            169466.666667  209366.666667  0.809425
          Casper           198033.333333  196533.333333  1.007632
          Cheyenne         182833.333333  193533.333333  0.944712
        

In [11]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    gdplev = pd.read_excel('gdplev.xls', skiprows = 219, usecols = [4,6], names = ['yq', 'gdp_09'])
    different = False
    better = ""
    p = 0
    
    # keep only housing price columns corresponding to recession start & recession bottom
    recession_housing = convert_housing_data_to_quarters()[[get_recession_start(), get_recession_bottom()]]
    
    # add third column - a ratio of the two (should be a smaller number for univ towns under current hypothesis - 
    #     they are less affected by recession ==> they keep growing)
    recession_housing['ratio'] = recession_housing.iloc[:,0]/recession_housing.iloc[:,1]
    
    # map state initials to full state names & group multiindex
    recession_housing = recession_housing.reset_index()
    recession_housing['State'] = recession_housing['State'].map(states)
    recession_housing = recession_housing.set_index(['State','RegionName']).groupby(level=[0,1]).mean()
    
    # split dataset - university towns vs non-university towns
    univ_housing = recession_housing.loc[get_list_of_university_towns().set_index(['State','RegionName']).index.tolist()]
    non_univ_housing = recession_housing.loc[recession_housing.index - univ_housing.index]
    
    # run t-test & retain relevant info as follows - 
    #     set different = True if null hypothesis rejected with p<0.01
    #     set p = p-value returned by scipy.stats.ttest_ind()
    #     set better = "university town" or "non-university town", depending on which has a lower mean price ratio
    # return (different, p, better)
    
    
    
    
    return non_univ_housing

run_ttest()

TypeError: cannot perform __sub__ with this index type: <class 'pandas.indexes.multi.MultiIndex'>